# Лабораторная работа №1

## Задания

Самостоятельно написать код, реализующий искусственный нейрон с сигма-функцией активации, и возможность строить на его основе многослойные сети. Код должен также реализовывать градиентный спуск и обратное распространение ошибки.

На основе вашего кода:

1. Решить задачу  классификации датаcета Iris одним нейроном.
2. Решить задачу  классификации датаcета Iris одним  нейросетью из 2 слоев по 10 нейронов в слое.
3. Отрисовать разделяющую линию для обеих моделей. Сравнить метрики классификации.

## Реализация

### Библиотеки

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

### Сигма-функция

Используется в качестве функции активации

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def der_sigmoid(x):  # производная сигма-функции
    return sigmoid(x) * (1 - sigmoid(x))

### Функция ошибок

Здесь будет использоваться MSE (Mean Square Error)

In [4]:
def mse(y_true, y_pred):
    return ((y_true - y_pred)**2).mean()

In [ ]:
def der_mse(y_true, y_pred):  # производная функции ошибок
    return -2 * (y_true - y_pred).mean()

### Слой нейронной сети

Хранит матрицу весов нейронов, матрицу сдвигов нейронов, матрицу ошибок и вывод нейронов

In [5]:
class Layer:
    def __init__(self, neuron_count):
        self.neuron_count = neuron_count
        self.weights = None  # матрица весов нейронов W x N, где N - кол-во нейронов в слое, а W - кол-во входных данных в слой
        self.biases = None  # вектор свдигов нейронов N, где N - кол-во нейронов в слое
        self.output = None  # вектор вывода N
        self.error = None  # вектор ошибок, нужен для обратного распространения ошибки в градиентном спуске

    
    def setup_layer(self, input_count):
        self.weights = np.random.rand(input_count, self.neutron_count)  # веса по умолчанию
        self.biases = np.random.rand(self.neutron_count)  # сдвиг по умолчанию

    
    def input_output(self, input_values):
        self.output = sigmoid((self.input_values @ self.weights) + self.biases)  # вектор вывода, здесь F(XW + B), где X - вектор входных данных, W - матрица весов, B - вектор сдвигов

        return self.output

### Нейронная сеть

Сеть состоит из слоёв, здесь определяется её поведение

In [6]:
class NeuralNetwork:
    def __init__(self, layers=[]):
        self.layers = layers

    
    def back_propogation(self, y_train):  # обратное распространение ошибки
        for layer in range(len(self.layers) - 1, -1, -1):  # идём в обратном порядке, в этом суть метода
            cur_layer = self.layers[layer]
            
            if layer == len(self.layers) - 1:
                # Для выходного слоя вектор ошибок будет выглядеть так: L'(t, o) * f'(o), где:
                # L' - производная функции ошибок, f' - производная функции активации
                # o - вывод значений нынешнего слоя, t - истинный вывод
                
                cur_layer.error_weight = der_mse(y_train, cur_layer.output) * der_sigmoid(cur_layer.output)

            else:
                # Для остальных слоёв вектор ошибок будет такой: DT * f'(o), где:
                # D - вектор ошибок следующего слоя, T - транспонированная матрица весов следующего слоя
                # f' - производная функции активации, o - вывод значений нынешнего слоя
                
                next_layer = self.layers[layer + 1]
                cur_layer.error = (next_layer.error @ next_layer.weights.transpose()) * der_sigmoid(cur_layer.output)

    
    def gradient_descent(self, X_train, learning_rate):  # градиентный спуск
        for layer in range(len(self.layers)):  # теперь идём по порядку, так называемый forward progogation     
            cur_layer = self.layers[layer]

            # Изменение сдвигов равно E*l, где l - скорость обучения, E - вектор ошибок

            delta_biases = cur_layer.error * learning_rate 

            # Изменение весов равно TE*l, где l - скорость обучения, E - вектор ошибок, T - транспонированная матрица вводных данных слоя

            delta_weights = None

            if layer == 0:
                delta_weights = (X_train.transpose() @ cur_layer.error) * learning_rate

            else:
                prev_layer = self.layers[layer - 1]
                delta_weights = (prev_layer.output.transpose() @ cur_layer.error) * learning_rate

            cur_layer.weights -= delta_weight
            cur_layer.biases -= delta_biases

        
    def fit(self, X_train, y_train, learning_rate=0.1, epochs=200, verbose=False, early_stopping=None):
        for layer in range(len(self.layers)):  # инициализация слоёв
            if layer == 0:
                self.layers[layer].setup_layer(len(X_train[0]))  # первый слой - входной, у него кол-во входных данных равно кол-ву вводимых атрибутов

            else:
                self.layers[layer].setup_layer(self.layers[layer-1].neuron_count)  # у каждого последующего слоя кол-во входных данных равно кол-ву нейронов на предыдущем слое
        
        last_error = None

        for epoch in range(epochs):  # начало обучения
            y_pred = []
            
            for value in range(len(X_train)):
                neuron_output = X_train[value]

                for layer in range(len(self.layers)):       
                    neuron_output = self.layers[layer].input_output(neuron_output)

                y_pred.append(neuron_output)  # значение последнего слоя и будет предсказанным значением
                
                self.backpropogation(y_train[value])  # считаем обратное распространение ошибки
                self.gradient_descent(X_train[value], learning_rate)  # выполняем градиентный спуск

            y_pred = np.array(y_pred)
            error = mse(y_train, y_pred)

            if verbose:
                print(f"Epoch {epoch+1} out of {len(X_train)}, current error is {error}")

            if early_stopping is not None:  # early stopping - минимальное значение разницы ошибок последних двух итераций, меньше которого обучение останавливается
                if last_error is not None:
                    if abs(error - last_error) < early_stopping:
                        if verbose:
                            print("Stopping early")
                        return

                last_error = error

    
    def predict(self, X):
        y_pred = []  # предсказанные значения

        for value in range(len(X)):            
            neuron_output = X[value]

            for layer in range(len(self.layers)):    
                neuron_output = self.layers[layer].input_output(neuron_output)

            y_pred.append(neuron_output)  # значение последнего слоя и будет предсказанным значением

        return np.array(y_pred)  # получили предсказание

### Датасет

In [7]:
df = load_iris()
X, y = df["data"], df["target"]
types = len(df["target_names"])  # т.к. решаем задачу классификации, то нужно знать, сколько всего возможных типов классификации вообще есть

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

### Преобразование датасета

X и y нужно преобразовать перед тем, как работать с ними

Предположим, для задачи классификации есть 4 возможных вывода (0, 1, 2, 3) и y выглядит так:

```
| y |
|---|
| 0 |
| 2 |
| 1 |
| 2 | 
| 3 |
| 1 |
```

Нейроны выдают значения от 0 до 1, и чтобы классификация прошла правильно, преобразуем данные в матрицу вхождений

```
| y |     |    y    |
|---|     |---------|
| 0 | --> | 1 0 0 0 |
| 2 | --> | 0 0 1 0 |
| 1 | --> | 0 1 0 0 |
| 2 | --> | 0 0 1 0 |
| 3 | --> | 0 0 0 1 |
| 1 | --> | 0 1 0 0 | 
```

В свою очередь, X нужно нормализировать перед работой с ней

In [9]:
def normalisation(X):
    for i in range(X.shape[1]):
        max_value, min_value = np.max(X[:,i]), np.min(X[:,i])
        X[:, i] =  (X[:, i] - min_value) / (max_value - min_value)

In [10]:
def hot_transform(y, types):
    N = len(y)
    
    transformed = np.zeros((N, types))

    for i in range(N):
        transformed[i][y[i]] = 1

    return transformed

In [11]:
normalisation(X_train)
X_train

array([[0.38888889, 0.41666667, 0.54237288, 0.45833333],
       [0.72222222, 0.45833333, 0.69491525, 0.91666667],
       [0.66666667, 0.45833333, 0.77966102, 0.95833333],
       [0.36111111, 0.29166667, 0.54237288, 0.5       ],
       [0.47222222, 0.375     , 0.59322034, 0.58333333],
       [0.27777778, 0.70833333, 0.08474576, 0.04166667],
       [0.55555556, 0.20833333, 0.66101695, 0.58333333],
       [0.22222222, 0.625     , 0.06779661, 0.08333333],
       [0.05555556, 0.125     , 0.05084746, 0.08333333],
       [0.5       , 0.375     , 0.62711864, 0.54166667],
       [0.19444444, 0.41666667, 0.10169492, 0.04166667],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.5       , 0.25      , 0.77966102, 0.54166667],
       [0.58333333, 0.45833333, 0.76271186, 0.70833333],
       [0.66666667, 0.45833333, 0.57627119, 0.54166667],
       [0.55555556, 0.54166667, 0.62711864, 0.625     ],
       [0.94444444, 0.41666667,

In [12]:
normalisation(X_test)
X_test

array([[0.28571429, 0.63157895, 0.06382979, 0.04166667],
       [0.5       , 0.42105263, 0.72340426, 0.70833333],
       [0.60714286, 0.57894737, 0.9787234 , 1.        ],
       [0.92857143, 0.42105263, 0.93617021, 0.625     ],
       [0.46428571, 0.42105263, 0.78723404, 0.70833333],
       [0.39285714, 0.84210526, 0.06382979, 0.08333333],
       [0.17857143, 0.57894737, 0.06382979, 0.16666667],
       [0.5       , 0.        , 0.55319149, 0.375     ],
       [0.57142857, 0.        , 0.65957447, 0.58333333],
       [0.82142857, 0.47368421, 0.74468085, 0.58333333],
       [0.21428571, 1.        , 0.0212766 , 0.        ],
       [0.75      , 0.42105263, 0.80851064, 0.91666667],
       [0.64285714, 0.52631579, 0.65957447, 0.58333333],
       [0.28571429, 0.89473684, 0.06382979, 0.125     ],
       [0.07142857, 0.47368421, 0.04255319, 0.04166667],
       [0.10714286, 0.73684211, 0.        , 0.        ],
       [0.67857143, 0.31578947, 0.68085106, 0.58333333],
       [0.32142857, 0.21052632,

In [13]:
y_train = hot_transform(y_train, types)
y_train

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0

In [14]:
y_test = hot_transform(y_test, types)
y_test

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

### Сеть из 1 слоя с 1 нейроном

In [15]:
model1 = NeuralNetwork([
    Layer(1),
    Layer(types)  # кол-во нейронов на выходном слою равно кол-ву типов классификации
])

In [16]:
model1.fit(X_train, y_train, epochs=1000, learning_speed=0.1, verbose=True)

In [17]:
y_pred1 = model1.predict(X_test)
y_pred1

array([[0.29472726, 0.1944138 , 0.5109827 ],
       [0.29423073, 0.19280598, 0.50994311],
       [0.29422767, 0.1927961 , 0.50993671],
       [0.29422791, 0.19279687, 0.50993721],
       [0.2942304 , 0.19280492, 0.50994243],
       [0.2943525 , 0.19319949, 0.51019817],
       [0.29480061, 0.19465207, 0.51113619],
       [0.29434097, 0.19316223, 0.51017403],
       [0.29425353, 0.19287961, 0.50999087],
       [0.29422866, 0.1927993 , 0.50993878],
       [0.29444281, 0.19349169, 0.51038729],
       [0.29422793, 0.19279694, 0.50993725],
       [0.29423005, 0.1928038 , 0.5099417 ],
       [0.29435818, 0.19321785, 0.51021005],
       [0.29644047, 0.20002706, 0.51456109],
       [0.29519957, 0.19595115, 0.51197055],
       [0.29423216, 0.19281059, 0.5099461 ],
       [0.29427936, 0.19296306, 0.51004498],
       [0.29443277, 0.19345919, 0.51036627],
       [0.2942279 , 0.19279684, 0.50993718],
       [0.29423274, 0.19281247, 0.50994733],
       [0.295818  , 0.1979758 , 0.51326248],
       [0.

### Сеть из 2 слоёв по 10 нейронов

In [18]:
model2 = NeuralNetwork([
    Layer(10),
    Layer(10),
    Layer(types)  # кол-во нейронов на выходном слою равно кол-ву типов классификации
])

In [19]:
model2.fit(X_train, y_train, epochs=1000, learning_speed=0.1, verbose=True)

Epoch 1 out of 1000, current max error is 0.6665710660206586
Epoch 2 out of 1000, current max error is 0.6665710803743669
Epoch 3 out of 1000, current max error is 0.6665710947216946
Epoch 4 out of 1000, current max error is 0.6665711090626464
Epoch 5 out of 1000, current max error is 0.6665711233972273
Epoch 6 out of 1000, current max error is 0.6665711377254416
Epoch 7 out of 1000, current max error is 0.6665711520472947
Epoch 8 out of 1000, current max error is 0.6665711663627907
Epoch 9 out of 1000, current max error is 0.6665711806719342
Epoch 10 out of 1000, current max error is 0.6665711949747304
Epoch 11 out of 1000, current max error is 0.6665712092711834
Epoch 12 out of 1000, current max error is 0.6665712235612983
Epoch 13 out of 1000, current max error is 0.6665712378450795
Epoch 14 out of 1000, current max error is 0.6665712521225321
Epoch 15 out of 1000, current max error is 0.6665712663936602
Epoch 16 out of 1000, current max error is 0.6665712806584688
Epoch 17 out of 1

In [20]:
y_pred2 = model2.predict(X_test)
y_pred2

array([[7.10453696e-08, 9.99852853e-01, 4.75057962e-23],
       [3.34168037e-07, 9.99642455e-01, 1.87527257e-21],
       [3.63624977e-07, 9.99624698e-01, 2.29172275e-21],
       [3.59459382e-07, 9.99627172e-01, 2.22988807e-21],
       [3.36312404e-07, 9.99641140e-01, 1.90396606e-21],
       [1.34220490e-07, 9.99788076e-01, 2.15095135e-22],
       [6.82348009e-08, 9.99856220e-01, 4.31649755e-23],
       [1.46949774e-07, 9.99776775e-01, 2.66714135e-22],
       [2.55686380e-07, 9.99693334e-01, 9.93286416e-22],
       [3.50240177e-07, 9.99632689e-01, 2.09650379e-21],
       [1.00151458e-07, 9.99820831e-01, 1.07338268e-22],
       [3.59126438e-07, 9.99627370e-01, 2.22498801e-21],
       [3.38542234e-07, 9.99639778e-01, 1.93407092e-21],
       [1.30814548e-07, 9.99791177e-01, 2.02362781e-22],
       [5.56683925e-08, 9.99872059e-01, 2.66244268e-23],
       [6.04493625e-08, 9.99865869e-01, 3.23763333e-23],
       [3.25949618e-07, 9.99647525e-01, 1.76763014e-21],
       [2.06337655e-07, 9.99728